<a href="https://colab.research.google.com/github/royn5618/Medium_Blog_Codes/blob/master/GenAI_for_NLP_Systems/nlp_for_social_media_analytics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

Blog Link:

In [ ]:
! pip install --upgrade "httpx==0.27.2"

In [ ]:
import openai
openai.__version__

'1.54.5'

In [ ]:
from openai import OpenAI
client = OpenAI(api_key=api_key)

In [ ]:
def get_response(prompt, ret=False):
  completion = client.chat.completions.create(
    model="gpt-4o-mini",
    temperature=0,
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {
            "role": "user",
            "content": prompt
        }
    ]
  )
  if ret:
    return completion.choices[0].message.content
  print(completion.choices[0].message.content)

# Text Cleaning and Standardising

Below I have provided Tweets from Natural Disasters Kaggle Challenge which has the typical characteristics of social media texts - phonetic spellings, hashtags, smileys, hyper-punctuations. In this section, I will demonstrate how we can use GPT to correct spellings and grammatical mistakes, plus standardise texts for ease of further text modeling.

## Basic Prompt

In [ ]:
# FEW SHOT

prompt = """ Following are a few tweet samples. Can you standardise all text with correct spelling and grammar.
---
Text: #RockyFire Update => California Hwy. 20 closed in both directions due to Lake County fire - #CAfire #wildfires
Text: What a goooooooaaaaaal!!!!!!
Text: My car is so fast
Text: London is cool ;)
Text: #flood #disaster Heavy rain causes flash flooding of streets in Manitou, Colorado Springs areas
Text: I'm afraid that the tornado is coming to our area.
---
"""

In [ ]:
 get_response(prompt)

Here are the standardized versions of the tweets with correct spelling and grammar:

1. **#RockyFire Update: California Highway 20 is closed in both directions due to a fire in Lake County. #CAfire #wildfires**
2. **What a goal!**
3. **My car is very fast.**
4. **London is cool.**
5. **#Flood #Disaster: Heavy rain causes flash flooding of streets in the Manitou and Colorado Springs areas.**
6. **I'm afraid that a tornado is coming to our area.**


## Improved Prompt

Note how GPT has interpreted some of the hashtags and has not addressed the emoji at all. This is where our "GenAI's Law of Specification" comes in. We need to state how each element should be treated explicitly. Example below:

In [ ]:
# FEW SHOT

prompt = """ Following are a few tweet samples. Can you standardise all text with correct spelling and grammar.
Preserve hashtags as they are. Remove smileys. Expand contractions.
---
Text: #RockyFire Update => California Hwy. 20 closed in both directions due to Lake County fire - #CAfire #wildfires
Text: What a goooooooaaaaaal!!!!!!
Text: My car is so fast
Text: London is cool ;)
Text: #flood #disaster Heavy rain causes flash flooding of streets in Manitou, Colorado Springs areas
Text: I'm afraid that the tornado is coming to our area.
---
"""

In [ ]:
 get_response(prompt)

Here are the standardized texts with correct spelling and grammar:

Text: #RockyFire Update: California Highway 20 is closed in both directions due to the Lake County fire - #CAfire #wildfires  
Text: What a goal!  
Text: My car is very fast.  
Text: London is cool.  
Text: #flood #disaster Heavy rain causes flash flooding of streets in Manitou and Colorado Springs areas.  
Text: I am afraid that the tornado is coming to our area.  


Notice how the texts are now cleaner.

# Text Classification

In this section I have demonstrated how to use GPTs for Text Classification.
There are ofcourse a few variations and more tailored approaches to your specific text classification needs.

I have enclosed two ways of instructing GPTs which can be further fine-tuned.

## Classify Disasters

In [ ]:
# FEW SHOT LEARNING

prompt = """ Following are a few examples of texts and coresponding classes to be used for text classification:
---
Text: #RockyFire Update => California Hwy. 20 closed in both directions due to Lake County fire - #CAfire #wildfires | Class: 1 \n
Text: What a goooooooaaaaaal!!!!!! | Class: 0 \n
Text: My car is so fast | Class: 0 \n
Text: London is cool ;) | Class: 0 \n
Text: #flood #disaster Heavy rain causes flash flooding of streets in Manitou, Colorado Springs areas | Class: 1 \n
Text: I'm afraid that the tornado is coming to our area. | Class: 1 \n
---
Classify the following:
Text: There's an emergency evacuation happening now in the building across the street | Class:  \n
"""

In [ ]:
 get_response(prompt)

Class: 1


Not note how the above response if the full text and Class 1 at the end, the same prompt resulted in Class: 1 a while back. So again, we go back to GenAI's Law of Specification.

## Modified Prompt for Research

The classification is correct but how do we know that GPT has the right idea. So below is an example of including explanation.

Explanation is interesting (bear with me to my next post on sentiment analysis). When processing in large batches, explanation will help in discerning the rationale that sometime may differ from the objective of your use-case.

In [ ]:
# FEW SHOT LEARNING

prompt = """ Following are a few examples of texts and coresponding classes to be used for text classification:
---
Text: #RockyFire Update => California Hwy. 20 closed in both directions due to Lake County fire - #CAfire #wildfires | Class: 1 \n
Text: What a goooooooaaaaaal!!!!!! | Class: 0 \n
Text: My car is so fast | Class: 0 \n
Text: London is cool ;) | Class: 0 \n
Text: #flood #disaster Heavy rain causes flash flooding of streets in Manitou, Colorado Springs areas | Class: 1 \n
Text: I'm afraid that the tornado is coming to our area. | Class: 1 \n
---
Classify the following and include an explanation:
Text: There's an emergency evacuation happening now in the building across the street | Class:  \n
"""

In [ ]:
 get_response(prompt)

Class: 1

Explanation: The text describes a situation involving an "emergency evacuation," which indicates a serious and urgent event. This aligns with the characteristics of Class 1, which includes texts related to emergencies, disasters, or significant safety concerns. The mention of an evacuation suggests that there is a potential threat to safety, similar to the other examples in Class 1 that discuss fires and floods.


## Improved for integrating with applications

In [ ]:
# FEW SHOT LEARNING

prompt = """ Following are a few examples of texts and coresponding classes to be used for text classification:
---
Text: #RockyFire Update => California Hwy. 20 closed in both directions due to Lake County fire - #CAfire #wildfires | Class: 1 \n
Text: What a goooooooaaaaaal!!!!!! | Class: 0 \n
Text: My car is so fast | Class: 0 \n
Text: London is cool ;) | Class: 0 \n
Text: #flood #disaster Heavy rain causes flash flooding of streets in Manitou, Colorado Springs areas | Class: 1 \n
Text: I'm afraid that the tornado is coming to our area. | Class: 1 \n
---
Classify the following. Use this JSON format: {tweet: <>, class: <>, explanation: <>}. Produce only the JSON response in {}. Do not use quote on JSON element names.
Text: There's an emergency evacuation happening now in the building across the street
"""

In [ ]:
 response1 = get_response(prompt, ret=True)
 response1

'{tweet: "There\'s an emergency evacuation happening now in the building across the street", class: 1, explanation: "The text indicates a situation that involves an emergency evacuation, which is related to a disaster or urgent safety concern, thus it is classified as class 1."}'

In [ ]:
 response2 = get_response(prompt, ret=True)
 response2

'{tweet: "There\'s an emergency evacuation happening now in the building across the street", class: 1, explanation: "The text indicates a situation that involves an emergency evacuation, which is a serious event related to safety and potential disaster."}'

In [ ]:
 response3 = get_response(prompt, ret=True)
 response3

'{tweet: "There\'s an emergency evacuation happening now in the building across the street", class: 1, explanation: "The text indicates a situation that involves an emergency evacuation, which is related to a disaster or urgent safety concern."}'

Here I have prompted GPT to generate a response thrice with the same prompt.

Note how the exaplanation has changed but not the structure. This ensures that if you are connecting to another application based on this JSON output, your code is safeguarded to the sporadic changes of GPT.

## Hate Speech Detection

Data Source: https://www.kaggle.com/datasets/mrmorj/hate-speech-and-offensive-language-dataset

In [ ]:
prompt = """ Below are four tweets. Label them as - Hate Speech, Offensive or Neither. Include an explanation.

1. RT @mayasolovely: As a woman you shouldn't complain about cleaning up your house. &amp; as a man you should always take the trash out...
2. @UrKindOfBrand Dawg!!!! RT @80sbaby4life: You ever fuck a bitch and she start to cry? You be confused as shit
3. @T_Madison_x: The shit just blows me..claim you so faithful and down for somebody but still fucking with hoes!
4. @CASHandBOOBIES: I been kidnapped yo bitch
"""

In [ ]:
get_response(prompt)

1. **Offensive** - This tweet perpetuates gender stereotypes by suggesting that women should accept their role in household chores and that men have a specific duty regarding trash. While it may not be outright hate speech, it is offensive as it reinforces traditional gender roles and can be seen as belittling to women.

2. **Offensive** - This tweet contains crude language and discusses a sensitive topic (sexual encounters and emotional reactions) in a disrespectful manner. It objectifies women and trivializes their feelings, which can be considered offensive.

3. **Neither** - This tweet expresses frustration about infidelity but does not contain hate speech or offensive language. It reflects a personal opinion or observation without targeting a specific group or using derogatory terms.

4. **Offensive** - This tweet makes a disturbing claim about kidnapping, which is a serious crime. Even if it is meant humorously or hyperbolically, it is offensive as it trivializes the issue of kid

For context, the data says:

- Tweet 1: Neither
- Tweet 2: Offensive
- Tweet 3: Hate Speech
- Tweet 4: Offensive

So accuracy looks like: 1/4

But what is concerning are the first and the third labels. Source of the ground truth is crowdsourced and a voting system for Hate Speech, Offensive and Neither categories have been utilised to formulate the answers. As human, our brains are biased and many times we fail to think an unbiased view which falters our understanding of biases in the data. This right here is an example.

Check how this obvious offensive message is classsed as neither in the Training Data. Also, note how the third tweet is referenced as neither, however, this is 100% looks offensive to me as a woman.

Now the last one however did not look like offensive to me but GenAI is bringing a perpective here.

Hate speech, offensive language or vulgarity can sometimes be confusing and hence, while GenAI can definitely do a good job at identifying these with zero-shot learning, based on business needs and your definition of these terms - present more clarity to the instructions as I have done below.

In [ ]:
prompt = """ Below are four tweets. Label them as - Hate Speech, Offensive or Neither. Find the definitions below:

---
Hate Speech: Hate against a race, gender, ethnicity or a group.
Offensive: Any usage of fowl and inappropritate language
Neither: That does not quality into any of the above.
---
Tweets:
1. RT @mayasolovely: As a woman you shouldn't complain about cleaning up your house. &amp; as a man you should always take the trash out...
2. im grabbing a minute to post i feel greedy wrong
3. @UrKindOfBrand Dawg!!!! RT @80sbaby4life: You ever fuck a bitch and she start to cry? You be confused as shit
4. @T_Madison_x: The shit just blows me..claim you so faithful and down for somebody but still fucking with hoes!
5. @CASHandBOOBIES: I been kidnapped yo bitch
6. Does anyone have any useful advice/guidance for whether the COVID vaccine is safe whilst breastfeeding?
7. Women are excellent sources of confusion.
"""

In [ ]:
get_response(prompt)

Here are the labels for the tweets based on the provided definitions:

1. **Offensive** - The tweet contains inappropriate language and promotes traditional gender roles in a derogatory manner.
2. **Neither** - The tweet does not contain hate speech or offensive language; it expresses a personal feeling.
3. **Offensive** - The tweet contains inappropriate language and discusses a sensitive topic in a crude manner.
4. **Offensive** - The tweet contains inappropriate language and discusses infidelity in a crude way.
5. **Offensive** - The tweet contains inappropriate language and suggests a serious crime in a casual manner.
6. **Neither** - The tweet is a request for information and does not contain hate speech or offensive language.
7. **Hate Speech** - The tweet generalizes women negatively, implying they are confusing, which can be seen as derogatory towards women as a group.


In [ ]:
get_response(prompt)

Here are the labels for the tweets based on the provided definitions:

1. **Offensive** - The tweet contains inappropriate language and promotes traditional gender roles in a derogatory manner.
2. **Neither** - This tweet does not contain hate speech or offensive language; it expresses a personal feeling.
3. **Offensive** - The tweet contains inappropriate language and discusses a sensitive topic in a crude manner.
4. **Offensive** - The tweet uses inappropriate language and discusses infidelity in a crude way.
5. **Offensive** - The tweet contains inappropriate language and suggests a serious crime in a casual manner.
6. **Neither** - This tweet is a request for information and does not contain hate speech or offensive language.
7. **Hate Speech** - This tweet generalizes women in a negative way, implying they are confusing, which can be seen as derogatory towards women as a group.


In [ ]:
get_response(prompt)

Here are the labels for the tweets based on the provided definitions:

1. **Offensive**: The tweet contains inappropriate language and promotes traditional gender roles in a derogatory manner.
2. **Neither**: This tweet does not contain hate speech or offensive language; it expresses a personal feeling.
3. **Offensive**: The tweet uses inappropriate language and discusses a sensitive topic in a crude manner.
4. **Offensive**: The tweet contains inappropriate language and discusses infidelity in a crude way.
5. **Offensive**: The tweet uses inappropriate language and suggests a serious crime in a casual manner.
6. **Neither**: This tweet is a request for information and does not contain hate speech or offensive language.
7. **Hate Speech**: This tweet generalizes women negatively, implying they are confusing, which can be seen as derogatory towards women as a group. 

So the final labels are:
1. Offensive
2. Neither
3. Offensive
4. Offensive
5. Offensive
6. Neither
7. Hate Speech


## Mental Health Signal

Data Source: https://www.kaggle.com/datasets/infamouscoder/mental-health-social-media

In [ ]:
prompt = f""" Try and detect any signals of mental health concerns:

Below are 20 tweets from User 101202:
I was told when I was diagnosed with depression that I needed a family doctor. 2 years later, still can't find one.
@jephjacques HELLO YES THIS IS DOG
RT @gpsforthebrain: Ah, spring. #Halifax #ohcanada https://t.co/32BPe0W2zD
@AlexReekie Plus who wants to go out with a fish?
@emastaflash @HfxGinger Hey it's totally a thing.
@threehares Maybe Wednesday or Thursday, once I've recovered from the trauma of several days on the surgical floor :)
@threehares I'm doing double shifts for the next 2 days, I doubt I'll be in any shape to go out after. :(
@threehares I has no time, I is working all the shifts :(
Burger Week needs less burger photos shoved down my throat and more ACTUAL BURGERS shoved down my throat.
I'm already so sick of hearing about Burger Week. I GET IT, YOUR MEAT AND BREAD IS DELCIOUS.
Hahaha wtf Canadian Tire docked me $0.25 on my return for not bringing back the fucking garbage Canadian Tire money they gave me.
"HELLO YES THIS IS WARD CLERK
I NEED TO ORDER ALL THE COMMODES AND ALL THE LAB TESTS"
Work complaints I text to BF: "I couldn't get my re-rostering started on time because everybody started pooping and bleeding!"
Makeup-wearing friends: recommendations for a vanity/makeup mirror with halo lighting?
Happy first day of spring! https://t.co/rKawwvgc30
RT @22_Minutes: Winter is over in a few hours. Don't look outside and you'll believe it.
"Bf: ""What the hell is Wonderwall?""
Me: ""You'll know it when you hear it.""
Bf: *hears first two words of lyrics* ""Yeah I know this.""
It's a slow going. In the meantime, though, I can shred some Carly Rae Jepsen.
Trying to learn Oasis songs on ukulele so I can actually say, "Anyway, here's Wonderwall."
LED lights are bullshit.
What is the point of having a 7x magnified mirror with a light if you need to be LITERALLY AN INCH AWAY FROM THE MIRROR TO GET ANY LIGHT?!
Wearing my new yellow raincoat and floofy hair today. Bf takes one look &amp; goes, "OK, April O'Neil."
Thinking of renting myself out as a professional housewife. I just need to learn how to cook.
In related news, I have completed the transformation into my mother. Remember me fondly. RIP, my youth.
I actually managed to mop the floors without bf &amp; cat immediately tracking footprints all over it. IT'S A CHRISTMAS MIRACLE!
"""

In [ ]:
get_response(prompt)

Based on the tweets provided, there are a few signals that may indicate mental health concerns, particularly related to the user's experience with depression and stress. Here are some observations:

1. **Struggles with Healthcare Access**: The user mentions being diagnosed with depression and struggling to find a family doctor for two years. This could indicate feelings of frustration, helplessness, or a lack of support, which are common in individuals dealing with mental health issues.

2. **Work-Related Stress**: The user references working double shifts and feeling overwhelmed ("I has no time, I is working all the shifts"). This suggests a high level of stress and possibly burnout, which can be detrimental to mental health.

3. **Use of Humor as a Coping Mechanism**: The user employs humor in various tweets (e.g., about work complaints and Burger Week). While humor can be a healthy coping mechanism, it can also sometimes mask deeper feelings of distress or dissatisfaction.

4. **Exp

## Sentiment and Opinion Mining

Data Source:

In [ ]:
prompt = """ Select which emotion it reflects from the emotion_list.
emotion_list =[sadness, anger, fear, joy, love].

Text: i didnt feel humiliated | emotion: [1, 0, 0, 0, 0]
Text: im grabbing a minute to post i feel greedy wrong | emotion: [0, 1, 0, 0, 0]
Text: i left with my bouquet of red and yellow tulips under my arm feeling slightly more optimistic than when i arrived | emotion: [0, 0, 0, 1, 0]
Text: i find myself in the odd position of feeling supportive of | emotion: [0, 0, 0, 0, 1]
Text: i feel a little nervous i go to the gym | emotion:
"""

In [ ]:
get_response(prompt)

Based on the provided text "i feel a little nervous i go to the gym", the emotion it reflects is **fear**. 

So, the emotion would be represented as: 
emotion: [0, 0, 1, 0, 0]


# Working with Different Languages

Inherently social media is multilingual... predominantly English but has a humungous base of diverse lanaguages, written in their own scripts, or phonetic english. This is a big challenge in social media analytics that can be addressed much more accurately and efficiently in short span of development and execution time through GPT than traditional NLP systems.

## Language Detection and Translation

In [ ]:
# FEW SHOT LEARNING

prompt = """ Detect language and produce a comma separated output using the following format: text, language, translated_en
Use \n as line separator and do not use any numbers for counting texts.

---
1. De droom van D66 wordt werkelijkheid: COVID-19 superdodelijk voor ouderen
2. نعم، يمكنني الكتابة باللغة العربية. كيف يمكنني مساعدتك؟
3. আজকের দিনটা কেমন যাচ্ছে আপনার? 😊
---

Preserve hashtags and emojis.
"""

In [ ]:
get_response(prompt)

De droom van D66 wordt werkelijkheid: COVID-19 superdodelijk voor ouderen, Dutch, The dream of D66 is becoming reality: COVID-19 super deadly for the elderly  
نعم، يمكنني الكتابة باللغة العربية. كيف يمكنني مساعدتك؟, Arabic, Yes, I can write in Arabic. How can I help you?  
আজকের দিনটা কেমন যাচ্ছে আপনার? 😊, Bengali, How is your day going today? 😊  


## Transliteration

In [ ]:
prompt = f"""
Identify the language of the following texts in Indian languages and translate them to English:

1. Chobi mukti kom howar onyotomo karon chilo, nirbhorjogyo shilpir shonkote chilen projojokera.
2. તળાવ સારું લાગે છે, હું હવે તરવા જાઉં છું.
3. beriliyam oxaid ek akaarbanik yaugik hai.
4. durdarshi kala - ernst fuchs, pole laifole, michael boven
5. prashikshan, pralekhan, prakashan evam antarashtriya sampark ityadi.
6. நான் மற்றொரு மலையேற்றத்திற்காக மலைகளுக்குச் செல்கிறேன்

Use the following format:
text | language | translated text
"""

In [ ]:
get_response(prompt)

Here are the translations and identifications of the languages for the provided texts:

1. Chobi mukti kom howar onyotomo karon chilo, nirbhorjogyo shilpir shonkote chilen projojokera. | Bengali | The reason for the photo's liberation was that the projectors were dependent on the shortage of skilled artists.

2. તળાવ સારું લાગે છે, હું હવે તરવા જાઉં છું. | Gujarati | The pond looks nice, I am going to swim now.

3. beriliyam oxaid ek akaarbanik yaugik hai. | Hindi | Beryllium oxide is a non-organic compound.

4. durdarshi kala - ernst fuchs, pole laifole, michael boven | Hindi | Visionary art - Ernst Fuchs, Pole Laifole, Michael Boven.

5. prashikshan, pralekhan, prakashan evam antarashtriya sampark ityadi. | Hindi | Training, writing, publishing, and international relations, etc.

6. நான் மற்றொரு மலையேற்றத்திற்காக மலைகளுக்குச் செல்கிறேன் | Tamil | I am going to the mountains for another climb.


# Topic Modeling

## Keyword Extractions

In [ ]:
prompt = f""" Below is a set of 5 tweets. Identify and generate a list of pharma keywords in these tweets:
---
Does anyone have any useful advice/guidance for whether the COVID vaccine is safe whilst breastfeeding?
while deaths are closing in on the 300,000 mark... millions of people wait #PfizerBioNTech #Vaccine !
The first COVID-19 vaccine doses have arrived in Oregon.
Had my Covid-19 Vaccination, doing my bit helping to save the human race. Mega sore arm today as expected
This vaccine is special because it triggers an immune response without using the actual virus that causes covid-19
---
"""

In [ ]:
get_response(prompt)

Here is a list of pharma keywords identified in the tweets:

1. COVID vaccine
2. Breastfeeding
3. PfizerBioNTech
4. Vaccine
5. COVID-19
6. Vaccination
7. Immune response
8. Virus
9. Doses
10. Sore arm

These keywords relate to pharmaceuticals, vaccines, and health-related topics discussed in the tweets.


In [ ]:
prompt = f""" Below is a set of 5 tweets. Your task is to detect if the tweets have location information in them or not.
Use this format:
text | has_location
<tweet> | Y
<tweet> | N
---
Does anyone have any useful advice/guidance for whether the COVID vaccine is safe whilst breastfeeding?
while deaths are closing in on the 300,000 mark... millions of people wait #PfizerBioNTech #Vaccine !
The first COVID-19 vaccine doses have arrived in Oregon.
Had my Covid-19 Vaccination, doing my bit helping to save the human race. Mega sore arm today as expected :(
This vaccine is special because it triggers an immune response without using the actual virus that causes covid-19
---
"""

In [ ]:
get_response(prompt)

Does anyone have any useful advice/guidance for whether the COVID vaccine is safe whilst breastfeeding? | N  
while deaths are closing in on the 300,000 mark... millions of people wait #PfizerBioNTech #Vaccine ! | N  
The first COVID-19 vaccine doses have arrived in Oregon. | Y  
Had my Covid-19 Vaccination, doing my bit helping to save the human race. Mega sore arm today as expected :( | N  
This vaccine is special because it triggers an immune response without using the actual virus that causes covid-19 | N  


## Topic Detection

In [ ]:
prompt= f""" Below are 5 tweet samples. Each of them may be attached to the topics - 'climate' or 'covid'. If tweet is related with neither, use 'Other'.

---
Had my Covid-19 Vaccination, doing my bit helping to save the human race. Mega sore arm today as expected :(
What a goooooooaaaaaal!!!!!!
#flood #disaster Heavy rain causes flash flooding of streets in Manitou, Colorado Springs areas
@threehares Maybe Wednesday or Thursday, once I've recovered from the trauma of several days on the surgical floor :)
---

"""

In [ ]:
get_response(prompt)

1. Covid
2. Other
3. Climate
4. Other


## Trend Detection & Associated Sentiment

In [ ]:
prompt = f"""
Below is a batch of tweets.
---
"When battling with a Zombie, I sometimes recite a bone-chilling chorus.
#Bard #TTRPG"
"A well-rehearsed comedy is like a rebellious roar of an overdue thunderstorm.
#Bard #Actor #Musician #Magician"
"When the world is at war, and the people struggle to survive, not all are inclined to pursue endeavors deemed 'frivolous'.

But what if those endeavors aren't 'frivolous' at all?

Lilene, shy and musically inclined, will learn how useful it is.  #FFXIV #Elezen #ARR #Bard https://t.co/dBZpIsIiMR"
"Today we bring you the iconic stars of the DnD multiverse, the Bards!

In our class discussion we talk about how versatile these virtuosos can be, especially in the social tier of the game.

#dnd5e #bard"
"Overall, using AI to create art can expand the boundaries of creativity and allow artists to explore new directions in their work.  ðŸ§µ4/4   @OpenAI #ChatGPT
ðŸ‘€"
"Just played around with OpenAI's new #ChatGPT model and asked it to write a poem about embodied carbon in buildings. This is what I got...

Try out ChatGPT
https://t.co/vaov2Ttluo

#embodiedcarbon #poetry https://t.co/DbgO6Qz2Zq"
I asked #ChatGPT to write me a story about Sova and Jett falling in love... https://t.co/yyZNrFRTQs
OpenAI's new ChatGPT is very good at creative writing.  #openAI #chatGPT #poetry #Bitcoin https://t.co/yxMfXVjxDB
This is a great improvement! Unlike text-davinci-003, #ChatGPT understands the context from previous userâ€™s answers.
My mind is blown. I cannot believe how good ChatGPT from OpenAI is. I see this already replacing a lot of googling tasks I do, 100% right out of the box. I donâ€™t even know what to say anymore. I must be doing something wrong. Someone please break it for me! #ChatGPT
"Absolutely insane. I asked #chatGPT to generate a simple personal website.

It showed step by step on how to create it, then added HTML and CSS.

The output was used on @Replit. ðŸ¤¯ https://t.co/yLigabNMAO"
#ChatGPT is pretty mind-blowing. https://t.co/jXD5pGcTt9
---

Extract:

1. Most popoular Topic:
2. Sentiment: <positive/negative/neutral>
3. Opinion: <joy/sadness/anxious/angry/excitement>
4. Associated Hashtags:
"""

In [ ]:
get_response(prompt)

1. Most popular Topic: ChatGPT and its creative capabilities
2. Sentiment: Positive
3. Opinion: Excitement
4. Associated Hashtags: #ChatGPT, #openAI, #poetry, #embodiedcarbon, #dnd5e, #bard, #TTRPG, #Actor, #Musician, #Magician


In [ ]:
prompt = f"""
Below is a batch of tweets.
---
"When battling with a Zombie, I sometimes recite a bone-chilling chorus.
#Bard #TTRPG"
"A well-rehearsed comedy is like a rebellious roar of an overdue thunderstorm.
#Bard #Actor #Musician #Magician"
"When the world is at war, and the people struggle to survive, not all are inclined to pursue endeavors deemed 'frivolous'.

But what if those endeavors aren't 'frivolous' at all?

Lilene, shy and musically inclined, will learn how useful it is.  #FFXIV #Elezen #ARR #Bard https://t.co/dBZpIsIiMR"
"Today we bring you the iconic stars of the DnD multiverse, the Bards!

In our class discussion we talk about how versatile these virtuosos can be, especially in the social tier of the game.

#dnd5e #bard"
"Overall, using AI to create art can expand the boundaries of creativity and allow artists to explore new directions in their work.  ðŸ§µ4/4   @OpenAI #ChatGPT
ðŸ‘€"
"Just played around with OpenAI's new #ChatGPT model and asked it to write a poem about embodied carbon in buildings. This is what I got...

Try out ChatGPT
https://t.co/vaov2Ttluo

#embodiedcarbon #poetry https://t.co/DbgO6Qz2Zq"
I asked #ChatGPT to write me a story about Sova and Jett falling in love... https://t.co/yyZNrFRTQs
OpenAI's new ChatGPT is very good at creative writing.  #openAI #chatGPT #poetry #Bitcoin https://t.co/yxMfXVjxDB
This is a great improvement! Unlike text-davinci-003, #ChatGPT understands the context from previous userâ€™s answers.
My mind is blown. I cannot believe how good ChatGPT from OpenAI is. I see this already replacing a lot of googling tasks I do, 100% right out of the box. I donâ€™t even know what to say anymore. I must be doing something wrong. Someone please break it for me! #ChatGPT
"Absolutely insane. I asked #chatGPT to generate a simple personal website.

It showed step by step on how to create it, then added HTML and CSS.

The output was used on @Replit. ðŸ¤¯ https://t.co/yLigabNMAO"
#ChatGPT is pretty mind-blowing. https://t.co/jXD5pGcTt9
---

Extract topics and rank them based on popularity.
"""

In [ ]:
get_response(prompt)

Based on the provided tweets, here are the extracted topics ranked by popularity:

1. **ChatGPT and AI**: 
   - Multiple tweets discuss the capabilities of ChatGPT, including creative writing, generating personal websites, and its overall effectiveness. This topic appears to be the most popular, with several tweets highlighting user experiences and improvements.

2. **Bards in TTRPGs (Tabletop Role-Playing Games)**:
   - Several tweets focus on Bards, particularly in the context of Dungeons & Dragons (DnD) and Final Fantasy XIV (FFXIV). The discussions revolve around their versatility, roles in storytelling, and character development.

3. **Creative Writing and Poetry**:
   - There are tweets that mention using AI for poetry and creative writing, showcasing the intersection of technology and artistic expression. This topic is closely related to the ChatGPT discussions.

4. **Art and Creativity**:
   - One tweet touches on the use of AI in creating art and expanding creative boundaries,

# User Profiling

In [ ]:
import requests
r = requests.get('https://www.instagram.com/taylorswift/')
user_info = r.text[:128000]

In [ ]:
prompt = f"""
Create user profile using the following JSON format:
{{

    name: <name>,
    location: <location>,
    influencer_status: <influencer/none>,
    profession: <profession>,
    num_followers: <followers count on Insta>,
    num_following: <following count on Insta>,
    total_posts: <number of posts on Insta>,
    bio: <bio>.
    bio_sentiment: <positive/negative/neutral>,
    bio_opinion: <joy/sadness/anxious/angry/excitement>,
    associated_brand: <endorsing any brand, then names>,
    interests: <possible interests>,
    urls: <associated urls>

}}
Produce only JSON.
---
User info: {user_info}
"""

In [ ]:
get_response(prompt)

```json
{
    "name": "Taylor Swift",
    "location": "Nashville, TN",
    "influencer_status": "influencer",
    "profession": "Singer-Songwriter",
    "num_followers": 250000000,
    "num_following": 1000,
    "total_posts": 3000,
    "bio": "Lover of music, cats, and coffee. 🌟",
    "bio_sentiment": "positive",
    "bio_opinion": "joy",
    "associated_brand": ["Reputation", "Taylor Swift Merch"],
    "interests": ["Music", "Writing", "Traveling", "Fashion"],
    "urls": ["https://www.instagram.com/taylorswift/"]
}
```


# Hallucination

So if you believed that the response from the previous link actually worked for building the user profile, then you are believing in a hallucination of the GPT model.

See how I have added specific instructions:

1. Have a structure
2. Generate only the JSON
3. Provided user details as a response from the URL
4. Also I have temperature = 0 (which is supposed to make GPT as factual as possible)

This is why GPT is not alerting that this is a sample. However there are a few ways of avoiding it.

In [ ]:
prompt = f"""
Create user profile using the following JSON format:
{{

    name: <name>,
    location: <location>,
    influencer_status: <influencer/none>,
    profession: <profession>,
    num_followers: <followers count on Insta>,
    num_following: <following count on Insta>,
    total_posts: <number of posts on Insta>,
    bio: <bio>.
    bio_sentiment: <positive/negative/neutral>,
    bio_opinion: <joy/sadness/anxious/angry/excitement>,
    associated_brand: <endorsing any brand, then names>,
    interests: <possible interests>,
    urls: <associated urls>,
    content_derived_from_user_info: <YES - Referred from User Info provided below / NO - LLM Generated Sample>

}}
Produce only JSON.
---
User info: {user_info}
"""

In [ ]:
get_response(prompt)

```json
{
    "name": "Taylor Swift",
    "location": "Nashville, TN",
    "influencer_status": "influencer",
    "profession": "Singer-Songwriter",
    "num_followers": 250000000,
    "num_following": 1000,
    "total_posts": 3000,
    "bio": "Lover of music, cats, and coffee. 🌟",
    "bio_sentiment": "positive",
    "bio_opinion": "joy",
    "associated_brand": "Reputation, Folklore, Evermore",
    "interests": "Music, Writing, Philanthropy, Fashion",
    "urls": [
        "https://www.instagram.com/taylorswift/",
        "https://www.taylorswift.com/"
    ],
    "content_derived_from_user_info": "NO"
}
```


Below is the actual page source content, validated with the right information required.

In [ ]:
file = open("page_source.txt", "r")
content = file.read()
print(len(content))
file.close()

561735


In [ ]:
user_info = content[:-200000]
prompt = f"""
Create user profile using the following JSON format:
{{

    name: <name>,
    location: <location>,
    influencer_status: <influencer/none>,
    profession: <profession>,
    num_followers: <followers count on Insta>,
    num_following: <following count on Insta>,
    total_posts: <number of posts on Insta>,
    bio: <bio>.
    bio_sentiment: <positive/negative/neutral>,
    bio_opinion: <joy/sadness/anxious/angry/excitement>,
    associated_brand: <endorsing any brand, then names>,
    interests: <possible interests>,
    urls: <associated urls>,
    content_derived_from_user_info: <YES - Referred from User Info provided below / NO - LLM Generated Sample>

}}
Produce only JSON.
---
User info: {user_info}
"""

get_response(prompt)

```json
{
    "name": "Taylor Swift",
    "location": "Unknown",
    "influencer_status": "influencer",
    "profession": "Singer-Songwriter",
    "num_followers": 283000000,
    "num_following": 0,
    "total_posts": 675,
    "bio": "All’s fair in love and poetry... New album THE TORTURED POETS DEPARTMENT. Out now 🤍",
    "bio_sentiment": "positive",
    "bio_opinion": "joy",
    "associated_brand": "None",
    "interests": "Music, Writing, Performing",
    "urls": [
        "https://www.instagram.com/taylorswift/"
    ],
    "content_derived_from_user_info": "YES"
}
```


And with this I'd like to conclude that- Let LLMs speak because LLMs will try  to give you a hint if the response is factual or not. So take it and build around it!

**Thanks for visiting!**